In [1]:
import boto3
import os
import pandas as pd

In [2]:
os.environ["AWS_PROFILE"] = "men"   
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [3]:
bucket_name = "men-test-clean-data"

In [73]:
prefixes = [
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/",
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/pages/"
]

In [77]:
# Lista para almacenar archivos JSON encontrados
json_files = []

# Buscar archivos JSON en ambas carpetas
for prefix in prefixes:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                json_files.append(obj['Key'])



In [89]:
# Contador de coincidencias
count = 0
count_preescolar = 0
count_basica_primaria = 0
count_basica_secundaria = 0
count_media = 0


# Buscar archivos JSON en ambas carpetas
for prefix in prefixes:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                try:
                    # Leer JSON desde S3
                    json_obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
                    json_content = json.load(json_obj['Body'])  # Convertir JSON a diccionario

                    # Extraer "page_url" o "url"
                    page_url = json_content.get("metadataAttributes", {}).get("page_url", "")
                    url = json_content.get("metadataAttributes", {}).get("url", "")

                    count += 1
                    # Verificar si "basica-secundaria" está presente en alguno de los valores
                    if "preescolar-educacion-inicial" in page_url or "preescolar-educacion-inicial" in url:
                        count_preescolar += 1
                    elif "basica-primaria" in page_url or "basica-primaria" in url:
                        count_basica_primaria += 1
                    elif "basica-secundaria" in page_url or "basica-secundaria" in url:
                        count_basica_secundaria += 1
                    elif "/media" in page_url or "/media" in url:
                        count_media += 1


                except Exception as e:
                    print(f"Error procesando {obj['Key']}: {e}")

# Imprimir el resultado final
print(f"\nTotal JSON: {count}")
print(f"\n'preescolar-educacion-inicial': {count_preescolar}")
print(f"\n'basica-primaria': {count_basica_primaria}")
print(f"\n'basica-secundaria': {count_basica_secundaria}")
print(f"\n'media': {count_media}")


Total JSON: 330

'preescolar-educacion-inicial': 9

'basica-primaria': 37

'basica-secundaria': 22

'media': 103


In [91]:
# Lista para guardar resultados
results = []

# Buscar archivos JSON en ambas carpetas
for prefix in prefixes:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                try:
                    # Leer JSON desde S3
                    json_obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
                    json_content = json.load(json_obj['Body'])  # Convertir JSON a diccionario

                    # Extraer "page_url" o "url"
                    page_url = json_content.get("metadataAttributes", {}).get("page_url", "")
                    url = json_content.get("metadataAttributes", {}).get("url", "")

                    # Aplicar la lógica para asignar age_from y age_to
                    if "preescolar-educacion-inicial" in page_url or "preescolar-educacion-inicial" in url:
                        age_from, age_to = 3, 6
                    elif "basica-primaria" in page_url or "basica-primaria" in url:
                        age_from, age_to = 6, 11
                    elif "basica-secundaria" in page_url or "basica-secundaria" in url:
                        age_from, age_to = 11, 15
                    elif "/media" in page_url or "/media" in url:
                        age_from, age_to = 15, 17
                    else:
                        age_from, age_to = 0, 99  # Valor por defecto

                    # Actualizar los valores en el JSON
                    json_content["metadataAttributes"]["age_from"] = age_from
                    json_content["metadataAttributes"]["age_to"] = age_to

                    # Guardar los resultados
                    results.append({
                        "Archivo JSON": obj['Key'],
                        "page_url": page_url,
                        "age_from": age_from,
                        "age_to": age_to
                    })

                    # Subir el JSON actualizado a S3
                    s3_client.put_object(
                        Bucket=bucket_name,
                        Key=obj['Key'],
                        Body=json.dumps(json_content, indent=4)
                    )
                    print(f"Actualizado: {obj['Key']}")

                except Exception as e:
                    print(f"Error procesando {obj['Key']}: {e}")

# Imprimir resumen
print("\nResumen de asignaciones:")
for result in results:
    print(f"{result['Archivo JSON']} → page_url: {result['page_url']}, age_from: {result['age_from']}, age_to: {result['age_to']}")

Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/2.%20Anexo%202%20-%20Orientaciones%20para%20Las%20Familias%20-%20Directiva%205.pdf.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Fichero_DUA_1.pdf.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Manual_instalacion_PWA.pdf.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes%204%C2%B0%20y%205.pdf.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes_1%C2%B0a_3%C2%B0_IIPARTE.pdf.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Retos%20para%20gigantes_1%C2%B0a_3%C2%B0_IPARTE.pdf.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_MOVIL/documents/Secundaria%20activa%206%C2%B0%20y%207%C2%B0.pdf.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITA

In [ ]:
prefixes2 = [
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/",
    "contenidos_formativos/CONTENIDOS_PARA_APRENDER/"
]

In [4]:
def list_s3_objects(bucket_name, prefixes, max_keys=20):
    s3 = boto3.client('s3')
    
    for prefix in prefixes:
        print(f"Objects in {prefix}:")
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, MaxKeys=max_keys)
        
        if 'Contents' in response:
            for obj in response['Contents']:
                print(obj['Key'])
        else:
            print("No objects found.")
        
        print("-" * 40)

bucket_name = "men-test-clean-data"
prefixes2 = [
    "contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/",
    "contenidos_formativos/CONTENIDOS_PARA_APRENDER/"
]

list_s3_objects(bucket_name, prefixes2)

Objects in contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/:
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U02_L06/MG_L_G01_U02_L06.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U02_L06/MG_L_G01_U02_L06.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U03_L05/MG_L_G01_U03_L05.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U03_L05/MG_L_G01_U03_L05.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L01/MG_L_G01_U05_L01.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L01/MG_L_G01_U05_L01.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L02/MG_L_G01_U05_L02.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L02/MG_L_G01_U05_L02.txt.metadata.json
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L03/MG_L_G01_U05_L03.txt
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L03/MG

In [5]:
print(len("contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01"))
print(len("contenidos_formativos/CONTENIDOS_PARA_APRENDER/MenuMedia/g10"))

60
60


In [14]:
def list_unique_prefixes(bucket_name, prefixes):
    s3 = boto3.client('s3')
    unique_prefixes = set()
    
    for prefix in prefixes:
        print(f"Checking objects in {prefix}...")
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        if 'Contents' in response:
            for obj in response['Contents']:
                truncated_key = obj['Key'][:62]
                unique_prefixes.add(truncated_key)
        else:
            print(f"No objects found in {prefix}.")
        
    print("Unique first 60 characters of object keys:")
    for unique in sorted(unique_prefixes):
        print(unique)
    
    return unique_prefixes

list_unique_prefixes(bucket_name, prefixes2)


Checking objects in contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/...
Checking objects in contenidos_formativos/CONTENIDOS_PARA_APRENDER/...
Unique first 60 characters of object keys:
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G02_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G03_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G04_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G05_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G06_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G07_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G08_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G09_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G10_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G11_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G01_U
contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M

{'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G02_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G03_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G04_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G05_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G06_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G07_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G08_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G09_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G10_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G11_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G01_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G02_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G03_U',
 'contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_M_G04

In [15]:
import boto3
import json
import re

# Definir los rangos de edad basados en los grados
AGE_MAPPING = {
    "G01": (6, 7), "G02": (7, 8), "G03": (8, 9), "G04": (9, 10), "G05": (10, 11),
    "G06": (11, 12), "G07": (12, 13), "G08": (13, 14), "G09": (14, 15),
    "G10": (15, 16), "G11": (16, 17)
}

# Inicializar cliente S3
s3_client = boto3.client('s3')

# Lista para guardar resultados
results = []

# Función para asignar age_from y age_to basado en el nombre del archivo
def assign_age_from_filename(filename):
    match = re.search(r"[Gg](\d{1,2})", filename)
    if match:
        grade = f"G{int(match.group(1)):02d}"  # Asegurar formato GXX
        return AGE_MAPPING.get(grade, (0, 99))  # Valor por defecto si no se encuentra
    return (0, 99)


for prefix in prefixes2:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith('.json'):  # Filtrar solo JSON
                try:
                    # Leer JSON desde S3
                    json_obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
                    json_content = json.load(json_obj['Body'])  # Convertir JSON a diccionario

                    # Asignar edades según el nombre del archivo
                    age_from, age_to = assign_age_from_filename(obj['Key'])
                    
                    # Actualizar los valores en el JSON
                    json_content.setdefault("metadataAttributes", {})
                    json_content["metadataAttributes"]["age_from"] = age_from
                    json_content["metadataAttributes"]["age_to"] = age_to

                    # Guardar los resultados
                    results.append({
                        "Archivo JSON": obj['Key'],
                        "age_from": age_from,
                        "age_to": age_to
                    })

                    # Subir el JSON actualizado a S3
                    s3_client.put_object(
                        Bucket=bucket_name,
                        Key=obj['Key'],
                        Body=json.dumps(json_content, indent=4)
                    )
                    print(f"Actualizado: {obj['Key']}")

                except Exception as e:
                    print(f"Error procesando {obj['Key']}: {e}")

# Imprimir resumen
print("\nResumen de asignaciones:")
for result in results:
    print(f"{result['Archivo JSON']} → age_from: {result['age_from']}, age_to: {result['age_to']}")

Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U02_L06/MG_L_G01_U02_L06.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U03_L05/MG_L_G01_U03_L05.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L01/MG_L_G01_U05_L01.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L02/MG_L_G01_U05_L02.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L03/MG_L_G01_U05_L03.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U05_L05/MG_L_G01_U05_L05.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U06_L01/MG_L_G01_U06_L01.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/menu_L_G01_U06_L02/MG_L_G01_U06_L02.txt.metadata.json
Actualizado: contenidos_formativos/APRENDER_DIGITAL_LIGERA_WEB/m